# Detecção de bordas e limiares em fotografias de fósseis da Formação Ponta Grossa

**Visão Computacional e Percepção - TA1**

**Henrique Luiz Rieger - GRR20190357**


## Introdução
Este trabalho da disciplina de Visão Computacional e Percepção busca aplicar as técnicas básicas de manipulação de imagens vistas no começo do curso: aplicação de filtros, correções, limiares e detecção de bordas; utilizando como ferramenta a biblioteca OpenCV. Como aplicação prática, é proposta a utilização dessas técnicas em 54 fotografias de fósseis da Formação Ponta Grossa, pertencentes ao acervo do [LABPALEO-UFPR](http://www.labpaleo.ufpr.br). O propósito desta aplicação é testar a viabilidade de utilizar essas ferramentas como coletores de informação nos fósseis, a fim de criar um ambiente que permita a obtenção de características dos espécimes apresentados.

## Métodos aplicados
Para possibilitar a obtenção de caracteres nas fotografias de fósseis, diversas técnicas básicas de visão computacional foram aplicadas. Primeiramente, é aplicado um filtro bilateral, a fim de reduzir ruídos inerentes ao tipo de fotografia analisada. Depois, é aplicada uma correção de gama, de forma a tentar ressaltar alguns contornos mais suaves dos fósseis. Por fim, duas técnicas de separação do fóssil da matriz rochosa são paralelamente empregadas e seus resultados, comparados: um limiar (*threshold*) binário e o algoritmo de detecção de bordas de Canny.

## Implementação
Primeiramente, precisamos importar as bibliotecas que vamos usar para estes experimentos. A célula abaixo importa as bibliotecas NumPy, matplotlib, OpenCV (`cv2`), glob (para manipulação fácil de arquivos) e ipywidgets (para interação).

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import glob

import ipywidgets as widgets
from ipywidgets import interact, interactive, IntSlider, FloatSlider

Agora, vamos definir uma função para importar uma imagem como um vetor de números, utilizando a biblioteca OpenCV. A função recebe como entrada um caminho para o arquivo a ser aberto.

In [3]:
def get_img(path: str):
    img = cv.imread(path, cv.IMREAD_GRAYSCALE)
    return img

Em seguida, utilizando a biblioteca glob, podemos obter uma lista de todos os nomes e caminhos de arquivos `.png` presentes no diretório `img`. Uma pequena prévia destes caminhos é usada para verificar a corretude da função.

In [4]:
samples = glob.glob(r'img/*.png')
samples.sort()
samples[0:5]

['img/Catalogo_fosseis_PontaGrossa-10.png',
 'img/Catalogo_fosseis_PontaGrossa-101.png',
 'img/Catalogo_fosseis_PontaGrossa-102.png',
 'img/Catalogo_fosseis_PontaGrossa-104.png',
 'img/Catalogo_fosseis_PontaGrossa-105.png']

Vamos então definir algumas funções para encapsular as operações da biblioteca OpenCV em chamadas mais simples.

Primeiramente, definimos a função `bilateral_filter`, que devolve a imagem de entrada com um filtro bilateral aplicado. Esse filtro foi selecionado pois possui a excelente propriedade de eliminar ruídos de alta frequência enquanto mantém a consistência das bordas, sem borrá-las muito. Isto se fez necessário pois a matriz rochosa na qual o fóssil está inserido é bastante irregular, introduzindo muitas sombras e contornos pequenos que atrapalham na detecção das bordas do ser vivo preservado. Dessa forma, podemos eliminar boa parte dessa textura, permitindo que os algoritmos de detecção foquem em procurar apenas a informação desejada.

Abaixo, podemos ver um teste desta função aplicada a um fóssil de [[algum fóssil]]

In [7]:
def bilateral_filter(img, d, sigma):
    return cv.bilateralFilter(img, d, sigma, sigma)

def test_bilateral(sample, d, sigma):
    img = get_img(sample)
    bl = bilateral_filter(img, d, sigma)
    plt.imshow(bl, 'gray')
    
interactive(test_bilateral, sample=samples, d=(1, 10), sigma=(10, 250))

interactive(children=(Dropdown(description='sample', options=('img/Catalogo_fosseis_PontaGrossa-10.png', 'img/…

Agora, vamos definir a função `gamma_correction`, que realiza uma correção de gama para tentar iluminar ou escurecer, de maneira exponencial, a imagem. A ideia é revelar características dos espécimes que podem aparecer muito fracamente quanto em uma segunda tentativa de ofuscar ruídos causados pela textura da rocha, ressaltando também as bordas principalmente de fósseis carbonificados (mais escuros em relação à matriz). Caso não seja necessário, é sempre possível definir o valor do coeficiente gama para `1` e manter a imagem inalterada.

Abaixo, podemos ver um teste da correção de gama, aplicada a um fóssil de [[algum fóssil]].

In [8]:
def gamma_correction(img, gamma):
    invGamma = 1 / gamma

    table = [((i / 255) ** invGamma) * 255 for i in range(256)]
    table = np.array(table, np.uint8)

    return cv.LUT(img, table)

def test_gamma(sample, gamma):
    img = get_img(sample)
    gc = gamma_correction(img, gamma)
    plt.imshow(gc, 'gray')
    
interactive(test_gamma, sample=samples, gamma=FloatSlider(min=0.01, max=5, value=2.2))

interactive(children=(Dropdown(description='sample', options=('img/Catalogo_fosseis_PontaGrossa-10.png', 'img/…

In [9]:
def threshold(img, thresh):
    return cv.threshold(img, thresh, 255, cv.THRESH_BINARY)[1]
    
def test_threshold(sample, thresh):
    img = get_img(sample)
    tsh = threshold(img, thresh)
    fig, axes = plt.subplots(1, 2)
    axes[0].imshow(img, 'gray')
    axes[1].imshow(tsh, 'gray')
    
interactive(test_threshold, sample=samples, thresh=IntSlider(min=0, max=255, value=100))

interactive(children=(Dropdown(description='sample', options=('img/Catalogo_fosseis_PontaGrossa-10.png', 'img/…

In [10]:
def edges(img, minimum, maximum, aperture, l2_gradient):
    return cv.Canny(img, minimum, maximum, apertureSize = aperture, L2gradient=l2_gradient)

#     plt.subplot(121), plt.imshow(img, 'gray')
#     plt.xticks([]), plt.yticks([])
#     plt.title('Imagem original')

#     plt.subplot(122), plt.imshow(edges, 'gray')
#     plt.xticks([]), plt.yticks([])
#     plt.title('Detecção de bordas')

#     plt.show()
    
def test_edges(sample, minimum, maximum, aperture, l2_gradient):
    img = get_img(sample)
    edg = edges(img, minimum, maximum, aperture, l2_gradient)
    plt.imshow(edg, 'gray')
    
interact(test_edges, sample=samples, minimum=(0, 255), maximum=(0, 255), aperture=IntSlider(min=3, max=7, step=2, value=3), l2_gradient=False)

interactive(children=(Dropdown(description='sample', options=('img/Catalogo_fosseis_PontaGrossa-10.png', 'img/…

<function __main__.test_edges(sample, minimum, maximum, aperture, l2_gradient)>

In [19]:
def invert(img):
    return (255-img)

def generate(sample, d, sigma, gamma, thresh, edge_min, edge_max, aperture, l2_gradient):
    img = get_img(sample)
    bl = bilateral_filter(img, d, sigma)
    gamma = gamma_correction(bl, gamma)
    tsh = threshold(gamma, thresh)    
    edg = edges(gamma, edge_min, edge_max, aperture, l2_gradient)
    
    plt.subplot(231)
    plt.title('Imagem Original')
    plt.xticks([]), plt.yticks([])
    plt.imshow(img, 'gray')
    
    plt.subplot(232)
    plt.title('Filtro Bilateral')
    plt.xticks([]), plt.yticks([])
    plt.imshow(bl, 'gray')
    
    plt.subplot(233)
    plt.title('Gama aplicado')
    plt.xticks([]), plt.yticks([])
    plt.imshow(gamma, 'gray')
    
    plt.subplot(223)
    plt.title('Gama + Threshold (Invertido)')
    plt.xticks([]), plt.yticks([])
    plt.imshow(invert(tsh), 'gray')
    
    plt.subplot(224)
    plt.title('Gama + Canny')
    plt.xticks([]), plt.yticks([])
    plt.imshow(edg, 'gray')
    
interact(
    generate,
    sample=samples,
    d=(1, 10),
    sigma=(5, 250),
    gamma=FloatSlider(min=0.01, max=5, value=2.2),
    thresh=(0, 255),
    edge_min=IntSlider(min=0, max=255, value=100),
    edge_max=IntSlider(min=0, max=255, value=150),
    aperture=IntSlider(min=3, max=7, step=2, value=3),
    l2_gradient=True
)

interactive(children=(Dropdown(description='sample', options=('img/Catalogo_fosseis_PontaGrossa-10.png', 'img/…

<function __main__.generate(sample, d, sigma, gamma, thresh, edge_min, edge_max, aperture, l2_gradient)>